In [1]:
import os
try:
    print(file_path)
except:
    file_path = os.path.abspath('')
    os.chdir(os.path.dirname(file_path))
    print(file_path)

/root/code_Bao/stock_price_4_fun/notebook


In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
pd.set_option('display.max_columns', None)
import stumpy
import matplotlib.pyplot as plt
from datetime import datetime,timedelta
import numpy.testing as npt
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

import os
from vnstock import * #import all functions, including functions that provide OHLC data for charting
# from vnstock.chart import * # import chart functions

In [3]:
# df = stock_historical_data("HAH", "2020-01-01", "2023-10-10", "1D", "stock")
# fig = candlestick_chart(df, ma_periods=[50,200], show_volume=False, reference_period=300, figure_size=(15, 8), 
#                         title='VIC - Candlestick Chart with MA and Volume', x_label='Date', y_label='Price', 
#                         colors=('lightgray', 'gray'), reference_colors=('black', 'blue'))
# fig.show()

Sử dụng các tiêu chí lọc như sau để thiết lập tham số params.

- CANSLIM: epsGrowth1Year, lastQuarterProfitGrowth, roe, avgTradingValue20Day, relativeStrength1Month
- Giá trị: roe, pe, avgTradingValue20Day
- Cổ tức cao: dividendYield, avgTradingValue20Day
- Phá nền mua: avgTradingValue20Day, forecastVolumeRatio, breakout: 'BULLISH'
- Giá tăng + Đột biến khối lượng: avgTradingValue20Day, forecastVolumeRatio
- Vượt đỉnh 52 tuần: avgTradingValue20Day, priceBreakOut52Week: 'BREAK_OUT'
- Phá đáy 52 tuần: avgTradingValue20Day, priceWashOut52Week: 'WASH_OUT'
- Uptrend ngắn hạn: avgTradingValue20Day, uptrend: 'buy-signal'
- Vượt trội ngắn hạn: relativeStrength3Day,
- Tăng trưởng: epsGrowth1Year, roe, avgTradingValue20Day

In [4]:
def filter_stocks(param):
    df = stock_screening_insights (param, size=1700, drop_lang='vi')
    if len(df)!= 0:
        print(df.ticker.unique())
    return df

## canslim

In [17]:
canslim_params = {
    "exchangeName": "HOSE",
    "marketCap": (1_000, 200_000),
    "epsGrowth1Year": (25, 100),  # Minimum 1-year EPS growth
    "lastQuarterProfitGrowth": (25, 100),  # Minimum last quarter profit growth
    "roe": (20, 100),  # Minimum Return on Equity (ROE)
    # "avgTradingValue20Day": (50, 1000),  # Minimum 20-day average trading value
}
df = filter_stocks(canslim_params)
df.head()

""


In [18]:
params = {
            "exchangeName": "HOSE",
            "marketCap": (1_000, 200_000),
            'uptrend': 'buy-signal',
            'tcbsBuySellSignal':'BUY',
            # "avgTradingValue20Day": (10, 100_000),
            'priceNearRealtime': (15,80),
        }
df = stock_screening_insights (params, size=1700, drop_lang='vi')
df.head()

,ticker,companyName,marketCap,roe,stockRating,businessOperation,businessModel,financialHealth,alpha,beta,strongBuyPercentage,suddenlyHighVolumeMatching,forecastVolumeRatio,pe,pb,evEbitda,dividendYield,revenueGrowth1Year,revenueGrowth5Year,epsGrowth1Year,epsGrowth5Year,grossMargin,netMargin,doe,avgTradingValue5Day,avgTradingValue10Day,avgTradingValue20Day,relativeStrength3Day,relativeStrength1Month,relativeStrength3Month,relativeStrength1Year,totalTradingValue,priceNearRealtime,rsi14,foreignVolumePercent,tcRS,foreignBuySell20Session,numIncreaseContinuousDay,eps,volumeVsVSma5,volumeVsVSma10,volumeVsVSma20,volumeVsVSma50,priceGrowth1Week,priceGrowth1Month,prev1DayGrowthPercent,prev1MonthGrowthPercent,prev1YearGrowthPercent,prev5YearGrowthPercent,freeTransferRate,netCashPerMarketCap,netCashPerTotalAssets,profitForTheLast4Quarters,lastQuarterRevenueGrowth,secondQuarterRevenueGrowth,lastQuarterProfitGrowth,secondQuarterProfitGrowth,percent1YearFromPeak,percentAwayFromHistoricalPeak,percent1YearFromBottom,percentOffHistoricalBottom,exchangeName.en,industryName.en,uptrend.en,priceVsSMA5.en,priceVsSMA20.en,tcbsBuySellSignal.en,macdHistogram.en,priceVsSma10.en,priceVsSma50.en,bollingBandSignal.en,rsi14Status.en,hasFinancialReport.en,priceVsSMA100.en
0,HRC,,1500,2,3,1.4,2.3,2.8,-0.1,-0.1,78.6,80,0.3,149.3,3,-116.8,0.5,-3.09,1.1,-53.97,3.48,1.24,3.2,0.4,0,0,0,59,35,89,18,0.01,52.7,51.4,0.6,50,0,1,353,0.4,0.3,0.5,0.3,5.19,-4.18,6.14,-4.18,-7,61.66,44.9,-11.9,-22.4,10.7,2,-49.7,105.8,11.6,-14.5,-44.1,15.6,633,HOSE,Chemicals,Buy Signal,Price cross above SMA(5),Price cross above SMA(20),Buy,MACD Histogram < 0 and increase,Price cross above SMA(10),Price cross above SMA(50),Buy signal,Neutral,Yes,Price above SMA(100)


## Payback time

In [19]:
# https://www.youtube.com/watch?v=4YPyxfXML0A&t=1825s
paybacktime_params = {
    "exchangeName": "HOSE",
    "marketCap": (1_000, 200_000),
    'revenueGrowth5Year': (15,100),
    'pe':(10,20),
    "epsGrowth5Year": (10, 100),  # Minimum 1-year EPS growth
    "roe": (15, 100),  # Minimum Return on Equity (ROE)
    
    'priceNearRealtime': (15,80),
#     "lastQuarterProfitGrowth": (2, 100),  # Minimum last quarter profit growth
    # "avgTradingValue20Day": (10, 2000),  # Minimum 20-day average trading value
#     "breakout": 'BULLISH',  # Only buy stocks when the market is in an uptrend
}
df = filter_stocks(paybacktime_params)
df.head()

['DCM' 'DHC' 'PNJ' 'TDM']


,ticker,companyName,marketCap,roe,stockRating,businessOperation,businessModel,financialHealth,alpha,beta,activeBuyPercentage,strongBuyPercentage,suddenlyHighVolumeMatching,forecastVolumeRatio,pe,pb,evEbitda,dividendYield,revenueGrowth1Year,revenueGrowth5Year,epsGrowth1Year,epsGrowth5Year,grossMargin,netMargin,doe,avgTradingValue5Day,avgTradingValue10Day,avgTradingValue20Day,relativeStrength3Day,relativeStrength1Month,relativeStrength3Month,relativeStrength1Year,totalTradingValue,priceNearRealtime,rsi14,foreignVolumePercent,tcRS,foreignBuySell20Session,numDecreaseContinuousDay,eps,volumeVsVSma5,volumeVsVSma10,volumeVsVSma20,volumeVsVSma50,priceGrowth1Week,priceGrowth1Month,prev1DayGrowthPercent,prev1MonthGrowthPercent,prev1YearGrowthPercent,prev5YearGrowthPercent,freeTransferRate,netCashPerMarketCap,netCashPerTotalAssets,profitForTheLast4Quarters,lastQuarterRevenueGrowth,secondQuarterRevenueGrowth,lastQuarterProfitGrowth,secondQuarterProfitGrowth,percent1YearFromPeak,percentAwayFromHistoricalPeak,percent1YearFromBottom,percentOffHistoricalBottom,exchangeName.en,industryName.en,priceVsSMA20.en,foreignTransaction.en,tcbsBuySellSignal.en,macdHistogram.en,priceVsSma50.en,rsi14Status.en,hasFinancialReport.en,priceVsSMA100.en,tcbsRecommend.en
0,DCM,,16623,16.8,4.7,4.3,4.0,5.0,-0.4,1.1,46.1,64.2,8.4,1.0,10.2,1.7,5.6,12.4,61.35,22.61,136.69,46.58,5.89,2.45,0.0,0,0,0,81,10,77,46,133.85,31.25,50.4,11.03,54,996,1,3050,1.4,1.2,0.9,0.8,4.69,-14.03,-0.48,-14.03,14.60,325.91,25,12.1,13.7,1614.7,-9.0,-19.4,-89.9,-69.4,-17.6,-25.1,52.0,587.2,HOSE,Chemicals,Price below SMA(20),Buy > Sell,Buy,MACD Histogram > 0 and increase,Price below SMA(50),Neutral,Yes,Price below SMA(100),NaN
1,DHC,,3377,17.3,4.3,3.8,2.8,4.4,-0.6,0.6,69.2,22.7,1.2,0.9,10.6,1.8,10.4,9.5,-5.51,37.18,-21.17,32.68,13.46,7.04,0.3,0,0,0,36,28,76,33,10.70,41.80,49.1,35.27,43,938,1,3929,1.0,1.1,0.9,0.6,6.36,-10.68,-0.36,-10.68,15.38,122.79,70,-12.4,-14.6,316.3,-18.1,-19.4,-13.3,-19.0,-14.6,-54.9,77.5,3049.7,HOSE,Basic Resources,Price below SMA(20),Buy > Sell,Buy,MACD Histogram > 0 and increase,Price below SMA(50),Neutral,Yes,Price below SMA(100),NaN
2,PNJ,,25256,20.3,4.0,4.1,4.0,4.4,0.3,0.6,48.7,25.5,5.9,0.7,13.9,2.7,10.9,2.3,73.31,25.28,75.96,11.61,17.31,3.66,0.2,0,0,0,17,52,54,33,33.30,76.50,52.7,49.00,39,-10,1,5506,0.7,0.9,1.0,0.9,3.66,-3.36,-0.65,-3.36,7.55,17.15,85,-4.9,-9.4,1805.9,-6.1,-17.4,0.4,-8.0,-18.0,-27.9,16.3,1695.2,HOSE,Personal & Household Goods,Price above SMA(20),Buy > Sell,Buy,MACD Histogram > 0 and increase,Price below SMA(50),Neutral,Yes,Price below SMA(100),NaN
3,TDM,,4065,15.3,3.8,3.8,3.4,4.2,0.5,0.6,8.3,94.2,1.7,0.0,12.8,1.9,11.5,0.0,14.84,19.44,-32.93,23.76,46.22,52.99,0.1,0,0,0,9,54,72,64,0.05,40.25,43.5,4.89,50,-170,1,3139,0.0,0.0,0.0,0.0,-2.07,-4.17,-0.98,-4.17,36.94,148.50,70,-5.3,-8.6,313.9,-13.1,6.3,4.1,8.0,-6.4,-6.0,65.8,613.9,HOSE,Utilities,Price below SMA(20),Buy < Sell,Sell,MACD Histogram < 0 and decrease,Price below SMA(50),Neutral,Yes,Price below SMA(100),Wait to buy


## Tang Truong

In [20]:
tangtruong_params = {
    "exchangeName": "HOSE",
    "marketCap": (1_000, 200_000),
    "epsGrowth1Year": (10, 100),  # Minimum 1-year EPS growth
    "roe": (15, 100),  # Minimum Return on Equity (ROE)
    'priceNearRealtime': (15,80),
    # "avgTradingValue20Day": (10, 2000),  # Minimum 20-day average trading value
}
df = filter_stocks(tangtruong_params)
df.head()

['ACB' 'CAV' 'CNG' 'CTG' 'DBD' 'GAS' 'GMD' 'HAH' 'HDB' 'IMP' 'LIX' 'MBB'
 'OPC' 'PGD' 'PHR' 'PNJ' 'REE' 'S4A' 'SAB' 'SCS' 'SHP' 'SMB' 'STB' 'TLG'
 'TMP' 'TPB' 'TRA' 'VFG' 'VGC' 'VHC' 'VIB' 'VPI']


,ticker,companyName,marketCap,roe,stockRating,businessOperation,businessModel,financialHealth,alpha,beta,activeBuyPercentage,strongBuyPercentage,suddenlyHighVolumeMatching,forecastVolumeRatio,pe,pb,evEbitda,dividendYield,revenueGrowth1Year,revenueGrowth5Year,epsGrowth1Year,epsGrowth5Year,grossMargin,netMargin,doe,avgTradingValue5Day,avgTradingValue10Day,avgTradingValue20Day,relativeStrength3Day,relativeStrength1Month,relativeStrength3Month,relativeStrength1Year,totalTradingValue,priceNearRealtime,rsi14,foreignVolumePercent,tcRS,foreignBuySell20Session,numIncreaseContinuousDay,numDecreaseContinuousDay,eps,volumeVsVSma5,volumeVsVSma10,volumeVsVSma20,volumeVsVSma50,priceGrowth1Week,priceGrowth1Month,prev1DayGrowthPercent,prev1MonthGrowthPercent,prev1YearGrowthPercent,prev5YearGrowthPercent,freeTransferRate,netCashPerMarketCap,netCashPerTotalAssets,profitForTheLast4Quarters,lastQuarterRevenueGrowth,secondQuarterRevenueGrowth,lastQuarterProfitGrowth,secondQuarterProfitGrowth,percent1YearFromPeak,percentAwayFromHistoricalPeak,percent1YearFromBottom,percentOffHistoricalBottom,exchangeName.en,industryName.en,priceVsSMA20.en,tcbsBuySellSignal.en,macdHistogram.en,priceVsSma50.en,rsi14Status.en,hasFinancialReport.en,priceVsSMA100.en,foreignTransaction.en,tcbsRecommend.en,heatingUp.en,dmiSignal.en,priceBreakOut52Week.en,priceVsSma10.en,sarVsMacdHist.en,uptrend.en,bollingBandSignal.en,priceVsSMA5.en
0,ACB,,88556,24.3,4.2,4.6,3.9,3.6,0.2,1.1,35.5,12.1,10.5,1.0,5.9,1.3,NaN,5.5,22.18,20.27,42.54,35.07,NaN,47.93,NaN,0,0,0,45,77,59,57,108.16,22.55,56.9,30.00,60,0,NaN,1.0,3839,1.3,1.2,1.1,0.9,1.12,-0.66,-1.10,-0.66,30.10,107.49,90.0,NaN,NaN,14909.0,18.8,15.9,12.6,-2.0,-7.6,-25.7,50.8,689.0,HOSE,Banks,Price above SMA(20),Strong buy,MACD Histogram > 0 and increase,Price above SMA(50),Neutral,Yes,Price above SMA(100),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,CAV,,3967,24.6,3.9,2.7,3.8,3.6,0.1,0.7,51.0,28.6,2.0,1.5,9.2,2.4,5.8,23.9,8.16,10.67,14.43,3.34,12.33,5.07,1.0,0,0,0,55,74,92,84,1.21,68.90,64.3,0.24,76,1,NaN,1.0,7500,1.5,2.8,4.4,4.1,3.45,1.62,-0.14,1.62,71.26,68.57,3.7,-39.0,-37.2,431.2,-9.0,-21.3,250.0,-11.0,-7.8,-4.7,83.1,459.5,HOSE,Industrial Goods & Services,Price above SMA(20),Strong buy,MACD Histogram > 0 and increase,Price above SMA(50),Neutral,Yes,Price above SMA(100),Buy > Sell,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,CNG,,1032,18.5,4.1,2.7,3.1,4.8,0.0,1.1,37.8,18.6,15.0,0.9,9.9,1.8,3.6,11.4,36.88,26.06,42.91,-3.70,9.09,3.78,0.1,0,0,0,76,37,62,81,3.93,28.90,51.2,13.46,64,169,NaN,1.0,2909,1.2,1.2,0.9,0.5,5.67,-4.93,-1.70,-4.93,65.99,44.72,45.0,27.3,23.5,102.1,-26.0,-29.8,65.3,-62.6,-21.0,-25.2,117.8,377.3,HOSE,Utilities,Price above SMA(20),Buy,MACD Histogram > 0 and increase,Price below SMA(50),Neutral,Yes,Price below SMA(100),Buy > Sell,Wait to buy,Overheated in previous trading session,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,CTG,,140568,16.0,4.0,3.1,3.8,3.4,0.3,1.3,53.0,27.8,16.4,1.7,7.8,1.2,NaN,0.0,20.62,14.47,19.07,17.68,NaN,22.40,NaN,0,0,0,58,73,40,54,135.86,29.50,55.2,27.18,56,-294,2.0,NaN,3773,2.1,1.9,1.9,1.1,4.42,-0.34,0.85,-0.34,20.65,75.06,15.0,NaN,NaN,18133.2,0.4,11.5,15.7,12.0,-12.3,-28.3,32.6,387.4,HOSE,Banks,Price above SMA(20),Buy,MACD Histogram > 0 and increase,Price below SMA(50),Neutral,Yes,Price below SMA(100),Buy > Sell,NaN,NaN,Buy signal,NaN,NaN,NaN,NaN,NaN,NaN
4,DBD,,4378,21.2,3.8,3.0,3.3,4.8,0.0,0.0,92.4,12.4,51.0,1.1,15.1,3.1,13.9,0.0,-0.24,2.29,28.78,2.65,48.55,16.34,0.0,0,0,0,13,70,93,81,3.15,58.50,53.4,12.60,64,-76,0.0,0.0,3866,2.6,0.9,0.5,0.6,0.00,-0.85,0.00,-0.85,63.59,130.58,80.0,-0.5,-1.2,289.3,4.4,22.9,24.5,25.1,-2.5,-2.0,81.4,182.4,HOSE,Health Care,Price below SMA(20),Neutral,MACD Histogram < 0 and increase,Price above SMA(50),Neutral,Yes,Price above SMA(100),Buy > Sell,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## my params

In [21]:
my_params = {
    "exchangeName": "HOSE,HNX",
    "marketCap": (1_000, 200_000),
    "roe": (10, 100),  # Minimum Return on Equity (ROE)
    'pe': (10,20),
    'priceNearRealtime': (10,100),
    # "avgTradingValue20Day": (100, 1000),  # Minimum 20-day average trading value
#     'uptrend': 'buy-signal',
    'macdHistogram': 'macdHistLT0Increase',
    'strongBuyPercentage': (20,100),
    'relativeStrength3Day': (50,100),
}
df = filter_stocks(my_params)
df.head()

['CVT' 'PGD']


,ticker,companyName,marketCap,roe,stockRating,businessOperation,businessModel,financialHealth,alpha,beta,activeBuyPercentage,strongBuyPercentage,suddenlyHighVolumeMatching,forecastVolumeRatio,pe,pb,evEbitda,dividendYield,revenueGrowth1Year,revenueGrowth5Year,epsGrowth1Year,epsGrowth5Year,grossMargin,netMargin,doe,avgTradingValue5Day,avgTradingValue10Day,avgTradingValue20Day,relativeStrength3Day,relativeStrength1Month,relativeStrength3Month,relativeStrength1Year,totalTradingValue,priceNearRealtime,rsi14,foreignVolumePercent,tcRS,foreignBuySell20Session,numIncreaseContinuousDay,numDecreaseContinuousDay,eps,volumeVsVSma5,volumeVsVSma10,volumeVsVSma20,volumeVsVSma50,priceGrowth1Week,priceGrowth1Month,prev1DayGrowthPercent,prev1MonthGrowthPercent,prev1YearGrowthPercent,prev5YearGrowthPercent,freeTransferRate,netCashPerMarketCap,netCashPerTotalAssets,profitForTheLast4Quarters,lastQuarterRevenueGrowth,secondQuarterRevenueGrowth,lastQuarterProfitGrowth,secondQuarterProfitGrowth,percent1YearFromPeak,percentAwayFromHistoricalPeak,percent1YearFromBottom,percentOffHistoricalBottom,exchangeName.en,industryName.en,priceVsSMA5.en,priceVsSMA20.en,tcbsBuySellSignal.en,macdHistogram.en,priceVsSma10.en,priceVsSma50.en,rsi14Status.en,hasFinancialReport.en,priceVsSMA100.en,foreignTransaction.en
0,CVT,,1207,11.1,3.4,2.3,3.0,2.4,-0.2,0.0,91.8,47.8,269.3,2.8,12.7,1.3,8.7,0.0,40.08,11.35,0.79,-11.39,18.21,3.31,2.1,0,0,0,52,26,51,6,0.16,33.8,38.9,0.51,34,0,1,NaN,2659,3.7,3.2,3.1,4.5,2.42,-5.59,2.74,-5.59,-26.28,92.37,48.9,-155.6,-56.2,97.6,-9.6,-3.3,-2.9,3.1,-26.3,-35.7,9.4,3694.3,HOSE,Construction & Materials,Price cross above SMA(5),Price below SMA(20),Sell,MACD Histogram < 0 and increase,Price cross above SMA(10),Price below SMA(50),Out of over sold,Yes,Price below SMA(100),NaN
1,PGD,,3584,25.7,3.9,3.1,3.5,3.6,-0.2,0.6,50.0,92.1,12.1,0.2,10.2,2.7,4.8,20.9,25.33,10.69,62.47,12.45,5.49,0.94,0.0,0,0,0,74,19,43,79,0.04,36.2,43.3,46.98,54,45,0,0.0,3549,0.4,0.3,0.2,0.1,-1.50,-11.92,0.00,-11.92,59.10,27.48,3.5,40.0,40.6,351.3,-13.6,-18.1,-77.2,-10.4,-29.9,-26.8,92.9,540.8,HOSE,Utilities,NaN,Price below SMA(20),Neutral,MACD Histogram < 0 and increase,NaN,Price below SMA(50),Neutral,Yes,Price below SMA(100),Buy > Sell


## Trend

In [22]:
trend_params = {
    "exchangeName": "HOSE,HNX",
    "marketCap": (1_000, 200_000),
    "epsGrowth5Year": (20, 100),  # Minimum 1-year EPS growth
#     "roe": (10, 100),  # Minimum Return on Equity (ROE)
#     'pe': (10,20),
#     'pb': (0,4),
    'priceNearRealtime': (10,100),
    'macdHistogram': 'macdHistLT0Increase',
    'strongBuyPercentage': (20,100),
    # "avgTradingValue20Day": (100, 100_000),  # Minimum 20-day average trading value
    # 'uptrend': 'buy-signal',
}
df = filter_stocks(trend_params)
df.head()

['BWE' 'TDP' 'VSH']


,ticker,companyName,marketCap,roe,stockRating,businessOperation,businessModel,financialHealth,alpha,beta,activeBuyPercentage,strongBuyPercentage,suddenlyHighVolumeMatching,forecastVolumeRatio,pe,pb,evEbitda,dividendYield,revenueGrowth1Year,revenueGrowth5Year,epsGrowth1Year,epsGrowth5Year,grossMargin,netMargin,doe,avgTradingValue5Day,avgTradingValue10Day,avgTradingValue20Day,relativeStrength3Day,relativeStrength1Month,relativeStrength3Month,relativeStrength1Year,totalTradingValue,priceNearRealtime,rsi14,foreignVolumePercent,tcRS,foreignBuySell20Session,numIncreaseContinuousDay,numDecreaseContinuousDay,eps,volumeVsVSma5,volumeVsVSma10,volumeVsVSma20,volumeVsVSma50,priceGrowth1Week,priceGrowth1Month,prev1DayGrowthPercent,prev1MonthGrowthPercent,prev1YearGrowthPercent,prev5YearGrowthPercent,freeTransferRate,netCashPerMarketCap,netCashPerTotalAssets,profitForTheLast4Quarters,lastQuarterRevenueGrowth,secondQuarterRevenueGrowth,lastQuarterProfitGrowth,secondQuarterProfitGrowth,percent1YearFromPeak,percentAwayFromHistoricalPeak,percent1YearFromBottom,percentOffHistoricalBottom,exchangeName.en,industryName.en,priceVsSMA5.en,priceVsSMA20.en,foreignTransaction.en,tcbsRecommend.en,tcbsBuySellSignal.en,macdHistogram.en,priceVsSma50.en,rsi14Status.en,hasFinancialReport.en,priceVsSMA100.en
0,BWE,,8450,15.5,3.8,3.9,3.8,3.2,0.4,0.8,55.6,33.1,6.3,0.8,12.1,1.8,9.3,0.0,11.70,14.17,-0.76,29.18,43.64,21.06,1.0,0,0,0,33,26,53,34,3.77,43.50,36.3,17.06,36,-1538,NaN,1.0,3599,0.9,0.5,0.5,0.5,-0.34,-4.40,-0.68,-4.40,5.09,141.42,30,-55.3,-41.2,694.3,-2.3,3.5,-10.0,3.6,-12.3,-27.2,38.2,214.8,HOSE,Utilities,Price cross below SMA(5),Price below SMA(20),Buy > Sell,Wait to buy,Sell,MACD Histogram < 0 and increase,Price below SMA(50),Neutral,Yes,Price below SMA(100)
1,TDP,,2273,4.7,3.3,3.2,3.0,1.8,0.8,0.6,100.0,69.4,0.1,0.9,60.8,2.8,16.8,0.0,33.10,44.14,7.98,33.71,8.58,1.06,2.9,0,0,0,11,27,50,65,4.59,30.95,42.8,0.10,38,-33,1.0,NaN,509,1.2,1.4,1.5,1.3,-0.16,-5.78,2.82,-5.78,35.75,NaN,45,-102.0,-63.1,38.5,25.2,13.1,-59.4,-84.4,-11.1,-11.1,38.7,202.0,HOSE,Industrial Goods & Services,Price cross above SMA(5),Price below SMA(20),NaN,NaN,Sell,MACD Histogram < 0 and increase,Price below SMA(50),Neutral,Yes,Price below SMA(100)
2,VSH,,10182,23.6,4.2,4.4,3.8,4.6,0.1,1.2,12.2,46.4,1.5,2.1,9.0,2.0,5.9,10.2,91.44,42.41,226.60,34.82,41.86,6.70,0.7,0,0,0,37,54,57,81,1.41,43.60,50.1,12.07,57,114,1.0,NaN,4852,3.2,1.7,1.6,1.1,1.40,-2.90,1.16,-2.90,56.44,181.01,20,-36.3,-38.7,1146.2,-41.4,-0.6,-88.4,1.9,-7.2,-6.0,105.0,1549.1,HOSE,Utilities,NaN,Price below SMA(20),NaN,NaN,Neutral,MACD Histogram < 0 and increase,Price below SMA(50),Neutral,Yes,Price below SMA(100)
